Insights to be drawn -
1. Get all Pokemons whose spawn rate is less than 5%
2. Get all Pokemons that have less than 4 weaknesses
3. Get all Pokemons that have no multipliers at all
4. Get all Pokemons that do not have more than 2 evolutions
5. Get all Pokemons whose spawn time is less than 300 seconds.
    Note - spawn time format is "05:32”, so assume “minute: second” format and perform the analysis.
6. Get all Pokemon who have more than two types of capabilities

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json

In [2]:
def list_of_int(list_):  
    try:
        data = list()
        for item in list_:
            data.append(round(item))
        return data
    except TypeError:
        return 0

In [3]:
def list_to_len(val: list):
    try:
        return len(val)
    except TypeError:
        return np.nan

In [4]:
def json_to_df():
    try:
        url = "https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json"

        data = pd.read_json(url)

        df = pd.json_normalize(data=data['pokemon'])

        df['avg_spawns'] = df['avg_spawns'].fillna(0).astype(np.int64)

        # column 'multipliers' - list of int
        df['multipliers'] = df['multipliers'].apply(lambda value: list_of_int(value))

        # Converting column 'spawn_time' to timedelta format
        df['spawn_time'] = df['spawn_time'].replace('N/A', '00:00')
        df['spawn_time'] = '00:' + df['spawn_time']
        df['spawn_time'] = pd.to_timedelta(df['spawn_time'])

        return df
    
    except Exception as e:
        raise Exception(e)

    
    

In [5]:
df = json_to_df()
df.head()

,id,num,name,img,type,height,weight,candy,candy_count,egg,spawn_chance,avg_spawns,spawn_time,multipliers,weaknesses,next_evolution,prev_evolution
0,1,001,Bulbasaur,http://www.serebii.net/pokemongo/pokemon/001.png,"[Grass, Poison]",0.71 m,6.9 kg,Bulbasaur Candy,25.0,2 km,0.690,69,0 days 00:20:00,[2],"[Fire, Ice, Flying, Psychic]","[{'num': '002', 'name': 'Ivysaur'}, {'num': '0...",NaN
1,2,002,Ivysaur,http://www.serebii.net/pokemongo/pokemon/002.png,"[Grass, Poison]",0.99 m,13.0 kg,Bulbasaur Candy,100.0,Not in Eggs,0.042,4,0 days 00:07:00,"[1, 2]","[Fire, Ice, Flying, Psychic]","[{'num': '003', 'name': 'Venusaur'}]","[{'num': '001', 'name': 'Bulbasaur'}]"
2,3,003,Venusaur,http://www.serebii.net/pokemongo/pokemon/003.png,"[Grass, Poison]",2.01 m,100.0 kg,Bulbasaur Candy,NaN,Not in Eggs,0.017,1,0 days 00:11:30,0,"[Fire, Ice, Flying, Psychic]",NaN,"[{'num': '001', 'name': 'Bulbasaur'}, {'num': ..."
3,4,004,Charmander,http://www.serebii.net/pokemongo/pokemon/004.png,[Fire],0.61 m,8.5 kg,Charmander Candy,25.0,2 km,0.253,25,0 days 00:08:45,[2],"[Water, Ground, Rock]","[{'num': '005', 'name': 'Charmeleon'}, {'num':...",NaN
4,5,005,Charmeleon,http://www.serebii.net/pokemongo/pokemon/005.png,[Fire],1.09 m,19.0 kg,Charmander Candy,100.0,Not in Eggs,0.012,1,0 days 00:19:00,[2],"[Water, Ground, Rock]","[{'num': '006', 'name': 'Charizard'}]","[{'num': '004', 'name': 'Charmander'}]"


### Get all Pokemons whose spawn rate is less than 5%

In [6]:
# Creating spawn rate column

df['spawn_rate'] = df['avg_spawns']/df['spawn_time'].dt.total_seconds()

In [8]:
df[df['spawn_rate'] < 0.05] [['name', 'spawn_rate']]

,name,spawn_rate
1,Ivysaur,0.009524
2,Venusaur,0.001449
3,Charmander,0.047619
4,Charmeleon,0.000877
5,Charizard,0.000000
...,...,...
140,Kabutops,0.000000
141,Aerodactyl,0.000704
142,Snorlax,0.000704
147,Dragonair,0.002789


### Get all Pokemons that have less than 4 weaknesses

In [9]:
df['weak_points'] = df['weaknesses'].apply(lambda x: list_to_len(x))

In [10]:
df[df['weak_points'] < 4].reset_index(drop=True) [['name', 'weaknesses']]

,name,weaknesses
0,Charmander,"[Water, Ground, Rock]"
1,Charmeleon,"[Water, Ground, Rock]"
2,Charizard,"[Water, Electric, Rock]"
3,Squirtle,"[Electric, Grass]"
4,Wartortle,"[Electric, Grass]"
...,...,...
97,Moltres,"[Water, Electric, Rock]"
98,Dratini,"[Ice, Dragon, Fairy]"
99,Dragonair,"[Ice, Dragon, Fairy]"
100,Mewtwo,"[Bug, Ghost, Dark]"


### Get all Pokemons that have no multipliers at all

In [11]:
df[df['multipliers'] == 0].reset_index(drop=True) [['name', 'multipliers']]

,name,multipliers
0,Venusaur,0
1,Charizard,0
2,Blastoise,0
3,Butterfree,0
4,Beedrill,0
...,...,...
76,Zapdos,0
77,Moltres,0
78,Dragonite,0
79,Mewtwo,0


### Get all Pokemons that do not have more than 2 evolutions

In [12]:
df['evolution'] = df['next_evolution'].apply(lambda x: list_to_len(x))

In [13]:
df[df['evolution'] <= 2].reset_index(drop=True) [['name']]

,name
0,Bulbasaur
1,Ivysaur
2,Charmander
3,Charmeleon
4,Squirtle
...,...
64,Magikarp
65,Omanyte
66,Kabuto
67,Dratini


### Get all Pokemons whose spawn time is less than 300 seconds

In [14]:
df['spawn_time_sec']=df['spawn_time'].dt.total_seconds()

In [15]:
df[df['spawn_time_sec'] < 300].reset_index(drop=True) [['name', 'spawn_time_sec']]

,name,spawn_time_sec
0,Squirtle,265.0
1,Blastoise,6.0
2,Metapod,131.0
3,Weedle,141.0
4,Kakuna,150.0
...,...,...
76,Articuno,0.0
77,Zapdos,0.0
78,Moltres,0.0
79,Mewtwo,0.0


### Get all Pokemon who have more than two types of capabilities

In [16]:
df['capabilities'] = df['type'].apply(lambda x: list_to_len(x))

In [17]:
df[df['capabilities'] > 2] [['name']]

,name


In [ ]:
df.drop(columns=['spawn_rate', 'weak_points', 'evolution', 'spawn_time_sec', 'capabilities'], inplace=True)